# Module 03: 1-D Diffusion Problem - Thawing Permafrost

### 1. Introduction and Background

In this notebook we are going to apply what we've learned about 1-dimensional diffusive processes to developing a model of how climate warming will effect permafrost. 


Below is the equation for 1-dimensional heat diffusion into a soil,
$$
\rho_s \cdot C_s \cdot \frac{\partial T_s}{\partial t} = \frac{\partial T_s}{\partial z}\left(k_s \cdot \frac{\partial T_s}{\partial z}\right)
$$

where $\rho_s$ (kg/m^3) is the bulk density of the soil, $C_s$ (J/(kg K)) is the heat capacity of the soil, $k_s$ (W/(m K)) is the thermal conductivity of the soil, $T_s$ is the temperature of the soil at some time $t$ and depth in the soil $z$. For the sake of simplicity, we're going to assume that the thermal conductivity is constant. This assumption is not strictly true, as $k_s$ changes with the amount and phase (frozen or liquid) of water in the soil. But it makes the math much simpler and is good starting point for a toy model. Under this assumption we can simplify the above equation to,

$$
\rho_s \cdot C_s \cdot \frac{\partial T_s}{\partial t} = k_s \frac{\partial T_s^2}{\partial^2 z}
$$

Going further, if we also assume that the bulk density ($\rho_s$) and heat capacity ($C_s$) are constant, then we can combine them with the thermal conductivity and reduce them to a single parameter that is often referred to as the _thermal diffusivity_ ($D_s$) and has dimensions of m^2/s, 

$$
D_s = \frac{k_s}{\rho_s \cdot C_s}
$$

Substituting into the above, we obtain the following governing equation that our model will solve, 

$$
\frac{\partial T_s}{\partial t} = D_s \frac{\partial T_s^2}{\partial^2 z}
$$

### 2. Applying a Numerical Solution

In this notebook, we will apply the forward time, centered time (FTCS) solution developed in Slingerland and Kump, an explicit method. We will divide our soil column into layers of equal thickness, specify boundary conditions at the top and bottom of our soil, as well as an initial condition and simulate the evolution of soil temperature over a 10 year period. Skipping the math of applying this method and go right to the finite difference solution. Using this method, we can solve for the soil temperature at the $(i+1)$th time and $j$th layer in the soil,

$$
T_{s,j}^{i+1} = s T_{s,j-1}^i + (1 - 2s)T_{s,j}^{i} + s T_{s,j+1}^i
$$

where $s$ combines the soil diffusivity, time step, and layer thickness as follows,

$$
s = D_s\frac{\Delta t}{\Delta z^2}
$$

For our __top boundary condition__, we will assume sinusoidal forcing of air temperature at the surface, with a period of one year. We will then need to specify the mean annual temperature around which the surface air temperature fluctuates, and the amplitude of annual fluctuations. In this notebook we will use realistic, but synthetic, values for each. 

For our __bottom boundary condition__, we will assume a constant temperature at the mean annual air temperature specified above. 

Our __initial condition__ will be a constant temperature (isothermal) at the mean annual air temperature. If interested you can copy this notebook and modify these values.

### 3. Initial Setup and Creating the Domain

Below, we set the values for our spatial domain, simulation period, soil properties, and boundary conditions. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Soil column properties
zs = 500.0 # Depth of soil [cm]
dz = 14.0   # Numerical model layer thickness [cm]

# Soil properties (obtained from https://doi.org/10.1029/2017JF004535)
Cs = 635.0 # Heat capacity of soil/permafrost [J/(kg K)]
ks = 0.25  # Thermal conductivity of soil/permafrost [W/(m K)]
rhos = 920.0 # Bulk density of permafrost [kg/m^3] 

# Time properties of simulation
t0 = 0
tf = 10*365         
dt = 0.25

# Boundary conditions
Ttopbase = -7.0
Ttopamp = 35.0
Tbottom = -7.0

#### 3.1 Setup the Soil Column Spatial Domain

In [ ]:
z = np.arange(dz/2, zs, dz)
Nz = z.size
print('Here are the locations of our cells: z = \n' + str(z))
print('The number of cells is: Nz = ',str(Nz))

#### 3.2 Setup the Simulation Time Period

In [ ]:
t = np.arange(t0,tf+dt,dt)
Nt = t.size
print('Here are the time steps: t = \n' + str(t))
print('The number of time steps: Nt = ',str(Nt))

#### 3.3 Calculate the Soil Diffusivity and Diffusion Number

In [ ]:
# Calculate diffusivity of permafrost soil
alpha = ks / (rhos * Cs)
print('Soil diffusivity = '+str(alpha)+' m^2/s')

# Convert alpha to units consistent with domain [cm^2/day]
alpha = alpha*(100.0**2)*(24.0*3600.0)
print('Soil diffusivity = '+str(alpha)+' cm^2/day')

s = alpha*dt/(dz**2)
print('Stability criterion '+str(s))

### 4. Setup Boundary and Initial Conditions

In [ ]:
Ttop = Ttopbase + (Ttopamp/2.0)*np.sin((2.0*np.pi/(365.0))*t)

plt.figure(figsize=(10.0,7.0))
plt.plot(t,Ttop)
plt.xlabel('Time [days]')
plt.ylabel('Air temperature [°C]')

Ts0 = Tbottom*np.ones((Nz,1))

### 5. Preallocate Memory for Temperature Simulation


In [ ]:
Ts = np.zeros((Nz,Nt))

### 6. Apply FTCS And Compute Temperatures

In [ ]:
for i in np.arange(Nt):
    for j in np.arange(Nz):
        
        # Check if we are at the initial time
        if(i==0): 
            if(j==0):
                T0zm1 = Ttop[i]
                T0z   = Ts0[j]
                T0zp1 = Ts0[j+1]
            elif((j>0) and (j<(Nz-1))):
                T0zm1 = Ts0[j-1]
                T0z   = Ts0[j]
                T0zp1 = Ts0[j+1]
            elif(j==(Nz-1)):
                T0zm1 = Ts0[j-1]
                T0z   = Ts0[j]
                T0zp1 = Tbottom
        else:
            if(j==0):
                T0zm1 = Ttop[i]
                T0z   = Ts[j,i-1]
                T0zp1 = Ts[j+1,i-1]
            elif((j>0) and (j<(Nz-1))):
                T0zm1 = Ts[j-1,i-1]
                T0z   = Ts[j,i-1]
                T0zp1 = Ts[j+1,i-1]
            elif(j==(Nz-1)):
                T0zm1 = Ts[j-1,i-1]
                T0z   = Ts[j,i-1]
                T0zp1 = Tbottom

        Tz = s*T0zm1 + (1.0 - 2.0*s)*T0z + s*T0zp1
                 
        Ts[j,i] = Tz


### 7. Plot Results, Analyze, and Interpret

First, let's create a pretty visualization of how temperature in the soil column changes with both depth below the surface and time. We will use the `pcolormesh` function in `matplotlib`. 

In [ ]:
## First off, let's change the font size for all of our plots to be more legible
plt.rcParams.update({'font.size': 16})

tt,zz = np.meshgrid(t,z)

plt.figure(figsize=(16,10)) # Create a figure and set the size

# Notice we're plotting -zz to have the top of the soil at the top and negative
# into the land surface
plt.pcolormesh(tt,-zz,Ts,cmap='inferno') 
plt.colorbar(label='$T_s$ [°C]')
plt.xlabel('Time [days]')
plt.ylabel('Soil depth [cm]')

Next, let's examine how the soil temperature evolves at several depths within the soil. Specifically, let's look at depths of 50, 100, 200, and 400 cm below the surface and plot how the temperature varies through time. Some important questions here:

* At what depth do we no longer cross above 0.0 °C in any given year? What can we say about the active layer thickness based on this? 
* What happens to the amplitude of the annual temperature fluctuations as we go deeper in the soil? 
* What happens to the timing of the peak temperature as we go deeper into the soil? 

In [ ]:
iz50 = (np.absolute(z-50.0)).argmin()
iz100 = (np.absolute(z-100.0)).argmin()
iz200 = (np.absolute(z-200.0)).argmin()
iz400 = (np.absolute(z-400.0)).argmin()

plt.figure(figsize=(16,10))
plt.plot(t,Ts[iz50,:],label='50 cm')
plt.plot(t,Ts[iz100,:],label='100 cm')
plt.plot(t,Ts[iz200,:],label='200 cm')
plt.plot(t,Ts[iz400,:],label='400 cm')
plt.xlabel('Time [Days]')
plt.ylabel('Soil Temperature [°C]')
plt.legend()

Finally, let's repeat the 3-D plot of temperature versus time and depth into the soil, but we'll mask values that are above 0 °C and plot them as blue (i.e., something that's very different than our previous colormap). This will allow us to see more clearly where the permafrost has thawed (i.e., it's temperature is above 0 °C) and how long it remains thawed. Some key questions we could address or concepts we can show by plotting the data this way include:

* What is the approximate thickness of the active layer based on this plot?
* How long throughout the year does the near surface (e.g., the top 10 cm) stay thawed each year?
* Do we notice any appreciable trends in active layer thickness at this time horizon? Why or why would you not expect one? 

In [ ]:
Ts_frozen = np.ma.masked_where(Ts>0.0,Ts)

plt.figure(figsize=(16,10))
current_cmap = plt.cm.get_cmap('inferno')
current_cmap.set_bad(color='blue')
plt.pcolormesh(tt,-zz,Ts_frozen,cmap=current_cmap)
plt.colorbar(label='$T_s$ [°C]')
plt.xlabel('Time [days]')
plt.ylabel('Soil depth [cm]')

### 8. A Final Couple Reflection Questions

As you review this notebook, think about the following:

* Why does the active layer seem to be thicker in the first year of the simulation? 
* If we changed the layer thickness $\Delta z$, do we have to change the time step $\Delta t$? Why? 